In [ ]:
!pip install thingspeak
!pip install tzdata

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
import thingspeak
import requests
from google.colab import drive
from datetime import timedelta, datetime

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=24ed27e5cadce2642eb5db80be162f9ef6b71e227080c657eed713f21e28985d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
drive.mount('/content/drive')

# Specify the path to your CSV file within Google Drive
data_path = '/content/drive/MyDrive/Weather Dataset/Srilanka_Weather_Dataset.csv'

# Load the CSV data into a pandas DataFrame
df = pd.read_csv(data_path)

# Check for missing values
print(df.isnull().sum())

Mounted at /content/drive
time                              0
temperature_2m (°C)               0
relative_humidity_2m (%)          0
weather_code (wmo code)           0
surface_pressure (hPa)            0
soil_moisture_0_to_7cm (m³/m³)    0
dtype: int64


In [ ]:
# Create a dictionary mapping WMO codes to descriptions
wmo_code_descriptions = {
    0: "Cloud development not observed or not observable",
    1: "Clouds generally dissolving or becoming less developed",
    2: "State of sky on the whole unchanged",
    3: "Clouds generally forming or developing",
    4: "Visibility reduced by smoke",
    5: "Haze",
    6: "Widespread dust in suspension in the air, not raised by wind at or near the station at the time of observation",
    7: "Dust or sand raised by wind at or near the station at the time of observation",
    8: "Well developed dust whirl(s) or sand whirl(s)",
    9: "Duststorm or sandstorm",
    10: "Mist",
    11: "Patches of shallow fog or ice fog at the station",
    12: "More or less continuous shallow fog or ice fog at the station",
    13: "Lightning visible, no thunder heard",
    14: "Precipitation within sight, not reaching the ground or the surface of the sea",
    15: "Precipitation within sight, reaching the ground or the surface of the sea",
    16: "Precipitation within sight, reaching the ground or the surface of the sea, near to station",
    17: "Thunderstorm, but no precipitation at the time of observation",
    18: "Squalls at or within sight of the station during the preceding hour or at the time of observation",
    19: "Funnel cloud(s)",
    20: "Drizzle (not freezing) or snow grains not falling as shower(s)",
    21: "Rain (not freezing) not falling as shower(s)",
    22: "Snow not falling as shower(s)",
    23: "Rain and snow or ice pellets, type (a) not falling as shower(s)",
    24: "Freezing drizzle or freezing rain not falling as shower(s)",
    25: "Shower(s) of rain",
    26: "Shower(s) of snow, or of rain and snow",
    27: "Shower(s) of hail, or of rain and hail",
    28: "Fog or ice fog",
    29: "Thunderstorm (with or without precipitation)",
    30: "Slight or moderate duststorm or sandstorm - has decreased during the preceding hour",
    31: "Slight or moderate duststorm or sandstorm - no appreciable change during the preceding hour",
    32: "Slight or moderate duststorm or sandstorm - has begun or has increased during the preceding hour",
    33: "Severe duststorm or sandstorm - has decreased during the preceding hour",
    34: "Severe duststorm or sandstorm - no appreciable change during the preceding hour",
    35: "Severe duststorm or sandstorm - has begun or has increased during the preceding hour",
    36: "Slight or moderate blowing snow generally low (below eye level)",
    37: "Heavy drifting snow generally low (below eye level)",
    38: "Slight or moderate blowing snow generally high (above eye level)",
    39: "Heavy drifting snow generally high (above eye level)",
    40: "Fog or ice fog at a distance at the time of observation, but not at the station during the preceding hour, the fog or ice fog extending to a level above that of the observer",
    41: "Fog or ice fog in patches",
    42: "Fog or ice fog, sky visible has become thinner during the preceding hour",
    43: "Fog or ice fog, sky invisible has become thinner during the preceding hour",
    44: "Fog or ice fog, sky visible no appreciable change during the preceding hour",
    45: "Fog or ice fog, sky invisible no appreciable change during the preceding hour",
    46: "Fog or ice fog, sky visible has begun or has become thicker during the preceding hour",
    47: "Fog or ice fog, sky invisible has begun or has become thicker during the preceding hour",
    48: "Fog, depositing rime, sky visible",
    49: "Fog, depositing rime, sky invisible",
    50: "Drizzle, not freezing, intermittent slight at time of observation",
    51: "Drizzle, not freezing, continuous slight at time of observation",
    52: "Drizzle, not freezing, intermittent moderate at time of observation",
    53: "Drizzle, not freezing, continuous moderate at time of observation",
    54: "Drizzle, not freezing, intermittent heavy (dence) at time of observation",
    55: "Drizzle, not freezing, continuous heavy (dence) at time of observation",
    56: "Drizzle, freezing, slight",
    57: "Drizzle, freezing, moderate or heavy (dence)",
    58: "Drizzle and rain, slight",
    59: "Drizzle and rain, moderate or heavy",
    60: "Rain, not freezing, intermittent slight at time of observation",
    61: "Rain, not freezing, continous slight at time of observation",
    62: "Rain, not freezing, intermittent moderate at time of observation",
    63: "Rain, not freezing, continuous moderate at time of observation",
    64: "Rain, not freezing, intermittent heavy at time of observation",
    65: "Rain, not freezing, continuous heavy at time of observation",
    66: "Rain, freezing, slight",
    67: "Rain, freezing, moderate or heavy",
    68: "Rain, or drizzle and snow, slight",
    69: "Rain, or drizzle and snow, moderate or heavy",
    70: "Intermittent fall of snow flakes slight at time of observation",
    71: "Continuous fall of snow flakes slight at time of observation",
    72: "Intermittent fall of snow flakes moderate at time of observation",
    73: "Continuous fall of snow flakes moderate at time of observation",
    74: "Intermittent fall of snow flakes heavy at time of observation",
    75: "Continuous fall of snow flakes heavy at time of observation",
    76: "Ice prisms (with or without fog)",
    77: "Snow grains (with or without fog)",
    78: "Isolated starlike snow crystals (with or without fog)",
    79: "Ice pellets, type (a)",
    80: "Rain shower(s), slight",
    81: "Rain shower(s), moderate or heavy",
    82: "Rain shower(s), violent",
    83: "Shower(s) of rain and snow mixed, slight",
    84: "Shower(s) of rain and snow mixed, moderate or heavy",
    85: "Snow shower(s), slight",
    86: "Snow shower(s), moderate or heavy",
    87: "Shower(s) of snow pellets or ice pellets, type (b), with or without rain or rain and snow mixed - slight",
    88: "Shower(s) of snow pellets or ice pellets, type (b), with or without rain or rain and snow mixed - moderate or heavy",
    89: "Shower(s) of hail*, with or without rain or rain and snow mixed, not associated with thunder - slight",
    90: "Shower(s) of hail*, with or without rain or rain and snow mixed, not associated with thunder - moderate or heavy",
    91: "Slight rain at time of observation - thunderstorm during the preceding hour but not at time of observation",
    92: "Moderate or heavy rain at time of observation - thunderstorm during the preceding hour but not at time of observation",
    93: "Slight snow, or rain and snow mixed or hail** at time of observation - thunderstorm during the preceding hour but not at time of observation",
    94: "Moderate or heavy snow, or rain and snow mixed or hail** at time of observation - thunderstorm during the preceding hour but not at time of observation",
    95: "Thunderstorm, slight or moderate, without hail**, but with rain and/or snow at time of observation - thunderstorm at time of observation",
    96: "Thunderstorm, slight or moderate, with hail** at time of observation - thunderstorm at time of observation",
    97: "Thunderstorm, heavy, without hail**, but with rain and/or snow at time of observation - thunderstorm at time of observation",
    98: "Thunderstorm combined with duststorm or sandstorm at time of observation - thunderstorm at time of observation",
    99: "Thunderstorm, heavy, with hail** at time of observation - thunderstorm at time of observation"
}

In [ ]:
# Convert 'time' column to datetime format (using ISO 8601 format)
df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%dT%H:%M')
df['time'] = df['time'].dt.tz_localize('GMT').dt.tz_convert('Asia/Colombo')

# Extract time-related features
df['hour'] = df['time'].dt.hour
df['dayofweek'] = df['time'].dt.dayofweek
df['month'] = df['time'].dt.month

# Diurnal Cycle Features
df['time_of_day'] = pd.cut(df['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])
# One-Hot Encode the time of day feature
df = pd.get_dummies(df, columns=['time_of_day'],prefix='',prefix_sep='')

# Sri Lankan Seasonal Features
def get_season(month):
  if month in [12, 1, 2]:
    return 'Maha'  # Northeast Monsoon
  elif month in [3, 4]:
    return 'Inter-Monsoon 1'  # First Inter-Monsoon
  elif month in [5, 6, 7, 8, 9]:
    return 'Yala'   # Southwest Monsoon
  else:
    return 'Inter-Monsoon 2'  # Second Inter-Monsoon

df['season'] = df['month'].apply(get_season)
# One-Hot Encode the season feature
df = pd.get_dummies(df, columns=['season'],prefix='',prefix_sep='')

# Lag features for next hour prediction
df['temperature_2m_prev_hour'] = df['temperature_2m (°C)'].shift(1)
df['relative_humidity_2m_prev_hour'] = df['relative_humidity_2m (%)'].shift(1)
df['surface_pressure_prev_hour'] = df['surface_pressure (hPa)'].shift(1)
df['soil_moisture_prev_hour'] = df['soil_moisture_0_to_7cm (m³/m³)'].shift(1)

# Lag features for next 24-hour prediction
df['temperature_2m_prev_day'] = df['temperature_2m (°C)'].shift(24)
df['relative_humidity_2m_prev_day'] = df['relative_humidity_2m (%)'].shift(24)
df['surface_pressure_prev_day'] = df['surface_pressure (hPa)'].shift(24)
df['soil_moisture_prev_day'] = df['soil_moisture_0_to_7cm (m³/m³)'].shift(24)

# Target variables
df['weather_code_next_hour'] = df['weather_code (wmo code)'].shift(-1)
df['weather_code_next_day'] = df['weather_code (wmo code)'].shift(-24)

# Drop rows with NaN values (created by the shifts)
df.dropna(inplace=True)

# Updated features list
features = ['temperature_2m (°C)', 'relative_humidity_2m (%)', 'surface_pressure (hPa)',
            'soil_moisture_0_to_7cm (m³/m³)', 'hour', 'dayofweek', 'month',
            'temperature_2m_prev_hour', 'relative_humidity_2m_prev_hour', 'surface_pressure_prev_hour', 'soil_moisture_prev_hour',
            'temperature_2m_prev_day', 'relative_humidity_2m_prev_day', 'surface_pressure_prev_day', 'soil_moisture_prev_day','Afternoon','Evening','Maha','Morning','Night','Yala']

X = df[features]
y = df[['weather_code_next_hour', 'weather_code_next_day']]

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy='mean')

X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Create a multi-output model
multi_output_model = MultiOutputClassifier(RandomForestClassifier(random_state=42)) # removed the hyperparameter tuning part.

# Train the ensemble model (this is usually very fast)
multi_output_model.fit(X_train, y_train)

#Make predictions on the test set
y_pred = multi_output_model.predict(X_test)

In [ ]:
#Evaluate the model performance
accuracy_next_hour = accuracy_score(y_test['weather_code_next_hour'], y_pred[:,0])
report_next_hour = classification_report(y_test['weather_code_next_hour'], y_pred[:,0])

accuracy_next_day = accuracy_score(y_test['weather_code_next_day'], y_pred[:,1])
report_next_day = classification_report(y_test['weather_code_next_day'], y_pred[:,1])

print("\nNext Hour Prediction Accuracy:", accuracy_next_hour)
print("Next Hour Classification Report:\n", report_next_hour)
print("\nNext Day Prediction Accuracy:", accuracy_next_day)
print("Next Day Classification Report:\n", report_next_day)


Next Hour Prediction Accuracy: 0.5024184900408405
Next Hour Classification Report:
               precision    recall  f1-score   support

         0.0       0.63      0.55      0.59     11123
         1.0       0.50      0.64      0.57     21288
         2.0       0.45      0.14      0.21     11286
         3.0       0.51      0.07      0.13      4914
        51.0       0.49      0.81      0.61     26116
        53.0       0.27      0.07      0.11      5866
        55.0       0.17      0.00      0.01      1736
        61.0       0.26      0.09      0.14      3194
        63.0       0.39      0.19      0.26      1990
        65.0       0.79      0.08      0.14       145

    accuracy                           0.50     87658
   macro avg       0.45      0.27      0.28     87658
weighted avg       0.48      0.50      0.45     87658


Next Day Prediction Accuracy: 0.4736475849323507
Next Day Classification Report:
               precision    recall  f1-score   support

         0.0      

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the training data
scaler.fit(X_train)

# Replace 'YOUR_CHANNEL_ID' and 'YOUR_READ_API_KEY' with your actual values
channel_id = '2468268'
read_api_key = 'JZU34XGC41T0AJCV'
write_api_key='7AP9CBEU9WJ59IO8'

# Construct the ThingSpeak API URL
url = f"https://api.thingspeak.com/channels/{channel_id}/feeds.json?api_key={read_api_key}&results=1"

# Fetch the latest feed using requests
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    feed = response.json()['feeds'][0]  # Get the latest feed entry

    # Extract only the live sensor values from the ThingSpeak feed (without lagged features)
    new_data = pd.DataFrame({
        'temperature_2m (°C)': [feed['field1']],
        'relative_humidity_2m (%)': [feed['field2']],
        'surface_pressure (hPa)': [feed['field3']],
        'soil_moisture_0_to_7cm (m³/m³)': [float(feed['field4'])/100],
        'hour': [pd.to_datetime(feed['created_at']).tz_convert('Asia/Colombo').hour],
        'dayofweek': [pd.to_datetime(feed['created_at']).tz_convert('Asia/Colombo').dayofweek],
        'month': [pd.to_datetime(feed['created_at']).tz_convert('Asia/Colombo').month]
    })
    # Ensure data types are correct
    new_data = new_data.astype(float)

    # Standardize features (IMPORTANT: Use a new scaler fitted on the columns of the new data)
    scaler_new_data = StandardScaler()
    new_data_scaled = scaler_new_data.fit_transform(new_data)
    new_data_scaled = pd.DataFrame(new_data_scaled, columns = new_data.columns)

    # Add the columns that are not in the new_data
    for col in features:
        if col not in new_data_scaled.columns:
            new_data_scaled[col] = 0

    # Make predictions
    predictions = multi_output_model.predict(new_data_scaled[features]) #use the correct features here

    next_hour_prediction = int(predictions[0][0])
    next_day_prediction = int(predictions[0][1])

    # Get descriptions
    next_hour_desc = wmo_code_descriptions.get(next_hour_prediction, "Unknown")
    next_day_desc = wmo_code_descriptions.get(next_day_prediction, "Unknown")

    # Rename columns for display
    new_data_display = new_data.rename(columns={
        'temperature_2m (°C)': 'Temperature (°C)',
        'relative_humidity_2m (%)': 'Humidity (%)',
        'surface_pressure (hPa)': 'Pressure (hPa)',
        'soil_moisture_0_to_7cm (m³/m³)': 'Soil Moisture (%)'
    })

    # Print live readings and prediction
    print("Live Readings from ThingSpeak:")
    print(new_data_display.to_string(index=False))  # Display renamed readings
    print("\nPredicted Weather Code (Next Hour):", next_hour_prediction)
    print("Weather Description (Next Hour):", next_hour_desc)
    print("\nPredicted Weather Code (Next Day):", next_day_prediction)
    print("Weather Description (Next Day):", next_day_desc)
    # Write the predicted weather code to ThingSpeak
    channel = thingspeak.Channel(id=channel_id, api_key=write_api_key)

    try:
        response = channel.update({
            'field5': next_hour_prediction,
            'field6': next_day_prediction
        })
        if response == 200:
            print("Predicted weather codes written to ThingSpeak successfully! Entry ID:", response.json()['entry_id'])
        else:
            print("Error writing to ThingSpeak:", response.json())
    except Exception as e:
        print("An error occurred while writing to ThingSpeak:", e)

else:
    print("Error fetching data from ThingSpeak:", response.status_code)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Live Readings from ThingSpeak:
 Temperature (°C)  Humidity (%)  Pressure (hPa)  Soil Moisture (%)  hour  dayofweek  month
             25.0          47.3      1006.62531               0.09  14.0        5.0    6.0

Predicted Weather Code (Next Hour): 0
Weather Description (Next Hour): Cloud development not observed or not observable

Predicted Weather Code (Next Day): 1
Weather Description (Next Day): Clouds generally dissolving or becoming less developed
An error occurred while writing to ThingSpeak: 'str' object has no attribute 'json'
